In [ ]:
import pandas as pd
#import xlwings as xw
#import xlsxwriter
import os
import pickle
import math
import numpy as np
import scipy.stats as st

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')   
os.chdir("/content/gdrive/My Drive/")


Mounted at /content/gdrive


In [ ]:

BASE_DIR =  'MIE490: Capstone/Data/chronological/'
# sanity check files in folder
os.listdir(BASE_DIR+'Male/')

['M_1996.xlsx',
 'M_2011.xlsx',
 'M_2007.xlsx',
 'M_2010.xlsx',
 'M_2006.xlsx',
 'M_1997.xlsx',
 'M_2001.xlsx',
 'M_2017.xlsx',
 'M_2000.xlsx',
 'M_2016.xlsx',
 'M_2003.xlsx',
 'M_2015.xlsx',
 'M_2019.xlsx',
 'M_2018.xlsx',
 'M_2014.xlsx',
 'M_2009.xlsx',
 'M_1998.xlsx',
 'M_2005.xlsx',
 'M_2013.xlsx',
 'M_1995.xlsx',
 'M_2012.xlsx',
 'M_2004.xlsx',
 'M_1999.xlsx',
 'M_2008.xlsx',
 'M_2002.xlsx']

## Prepare athlete data

In [ ]:
'''
Define the Athlete class to represent each athlete
'''
class Athlete:
    def __init__(self, fisCode, name, gender, country, beforeCompElo, elo, placement, birthdate, start_year, end_year, initial_level):
        self.fisCode = fisCode
        self.name = name
        self.gender = gender
        self.country = country
        self.beforeCompElo = beforeCompElo
        self.elo = elo
        self.placement = placement
        self.birthdate = birthdate
        self.start_year = start_year
        self.end_year = start_year
        self.initial_level = initial_level
        self.records = [None]*30

    def getInfo(self):
        return [self.fisCode, self.name, self.gender, self.birthdate, self.start_year, self.end_year, self.initial_level, self.country, self.elo]

    def updatePlacement(self, newPlacement):
        self.placement = newPlacement

    def updateElo(self, year):
        self.beforeCompElo = self.elo
        self.records[year-self.start_year] = self.elo
        self.end_year = year

    def __eq__(self, other):
        return self.fisCode == other.fisCode 

    def __hash__(self):
        return hash((self.name, self.fisCode))

    def __str__(self):
        return "{}: fisCode: {}, elo: {}, gender: {}, country: {}".format(self.name, self.fisCode, self.elo, self.gender, self.country)


In [ ]:
'''
A class of competition representation. Process all participated athletes in a list
'''

class Competition:
    def __init__(self):
        self.competitors = []

    def addCompetitor(self, athlete):
        self.competitors.append(athlete)

In [ ]:
'''
Process one competition data into a competition class object. 
Each athlete is updated with his/her rank in this competition.
After processing all athletes, their competition ranks  will be used to calculate Elo score. 
'''

def competitionFileParser(df, comp_year):
    # df is the dataframe that contains competition data for one competition
    # date = df.loc[0,'Race Date1']
    gender = df['Gender (Races!Sb)'].values[0]
    comp_level = df['Cup Type'].values[0]

    competition = Competition()
    for row in (df.index):
        fisCode = df.loc[row,'Fis Code']
        placement = df.loc[row,'Rank']
        birthdate = df.loc[row, 'Birthdate']
        # Initialize a new Athlete object
        if fisCode not in athlete_dict.keys():
            # NOTE: some athlete has no name...
            name = df.loc[row,'Athlete Name']
            country = df.loc[row,'Nation']
            # initilize different Elo scores for athlete based on the competition type
            if comp_level == 'WSC' or comp_level == 'WC':
                # Championship
                elo = 1500
            elif comp_level == 'NAC' or comp_level == 'WJC':
                elo = 1300
            elif comp_level == 'NC' or comp_level == 'NC/FI':
                elo = 1000
            else:  # just a catch, in case any error competition level type
                elo = 1300
                print(comp_level)
                print(df['Race Date'].values[0])
            athlete = Athlete(fisCode, name, gender, country, elo, elo, placement, birthdate, comp_year, comp_year, comp_level)
            athlete_dict[fisCode] = athlete
        # update athlete's rank in current competition
        else:
            athlete = athlete_dict.get(fisCode)
            athlete.updatePlacement(placement)

        # add athlete to the current competition class
        competition.addCompetitor(athlete)
        
    return competition


'''
Use the training data until the end_Date to calculate the Elo scores for all the 
athletes.

folderName: The path to either Male or Female competition data
'''

def train_system(folderName, end_date):
    files = os.listdir(folderName)
    # sort files in reverse chronological order 
    files.sort(reverse = False)
    # Go through each competition data
    for file in files:
        # discard qualification competition
        # if file == 'M_HP_QL.xlsx':
        #      continue
        end_year = int(str(end_date)[:4])
        file_year = int(file.split('_')[-1][:4])
        if file_year > end_year:
            continue 
        if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
            xls = pd.ExcelFile(folderName +'/' + file)
            sheet_names = xls.sheet_names
            ## sheet_names.sort(reverse = False)
            ## each sheet is a competition in that competition level
            for sheet in sheet_names:
                comp_date = int(sheet.split('_')[-1])
                comp_year = int(sheet.split('_')[-1][0:4])
                # ground truth ranking data starts from 20131205
                # use all data prior 20131205 as training data
                if comp_date < end_date:
                  df = xls.parse(sheet,header = 0)
                  # remove the athlete who DNS, DNF, DSQ
                  df = df[df['Status']== 'QLF']
                  
                  # process competition data 
                  competition = competitionFileParser(df, comp_year)
                  # run elo calculation for this competition,
                  # and update athletes' elo score, store their track of elo scores
                  calculateElo(competition.competitors, comp_year)
                  
            

In [ ]:
# ELO
# Background information - https://en.wikipedia.org/wiki/Elo_rating_system

# Elo system hyperparameters
_MAXSIZE = 80

_MINBUFFEREDELO = 1500
_MAXBUFFEREDELO = 2800
_KMAX = 25
_BUFFEREDELOSLOPE = -.28 / 1300
_EXPODENTIALRATE = 800
_WIN = 1.0
_LOSS = 0.0
_DEFAULT_K = 20
_MAXDENOM = 1/((_KMAX - _DEFAULT_K)/(3*_MAXSIZE))

def calculateElo(players, comp_year):
    N = len(players)
    # changing K multiplier depending on the size of the competition
    if N > _MAXSIZE:
        K = _DEFAULT_K
    else:
        k_slope = 3 * N / _MAXDENOM
        K = _KMAX - k_slope
    for i in range(0, N-1):
        player1 = players[i]
        # Players with higher elo receive lower K values for stability
        multi = 1
        if player1.beforeCompElo > _MINBUFFEREDELO and player1.beforeCompElo < _MAXBUFFEREDELO:
            multi = _WIN + (_BUFFEREDELOSLOPE) * (player1.beforeCompElo - _MINBUFFEREDELO)
        K = multi * K
        for j in range(i+1, N):
            player2 = players[j]
            
            # S is the player's actual score - (1 for win) - (0 for loss)
            if player1.placement < player2.placement:
                S = _WIN
            else:
                S = _LOSS
            change = player2.beforeCompElo - player1.beforeCompElo
            # Expected is the expected score based off each players elo
            expected_score = 1 / \
                (1.0 + math.pow(10.0, (change) / _EXPODENTIALRATE))
            player1.elo += K * (S - expected_score)
        # Making zero be the rating system's floor (optional)
        if player1.elo < 0:
            player1.elo = 0
    
    for player in players:
        player.updateElo(comp_year)


In [ ]:
### GLOBAL VARIABLE
# initialize the dictionary to store all athletes data
athlete_dict = {}

# train the Elo point system up until the end_date
train_system(BASE_DIR+'Male', end_date = 20131231)
train_system(BASE_DIR+'Female', end_date = 20131231)

len(athlete_dict)

4150

## Evaluate Athletes on Elo system

In [ ]:
'''
Sort the athlete dictionary and return a DataFrame with sorted athlete information
'''
def sort_athlete(athlete_dict):
    sorted_athlete = sorted(athlete_dict.items(), key=lambda x: x[1].elo, reverse=True)

    temp = []
    for fiscode, athlete in sorted_athlete:
      temp.append(athlete.getInfo())

    return pd.DataFrame(temp, columns = ['fiscode', 'name', 'gender', 'birthdate','First competition year','Last competition year', 'First competition level', 'country', 'elo'])

In [ ]:
### GLOBAL VARIABLE
# read the official ranking data
official_ranking_folder = 'MIE490: Capstone/Data/Ranking'
official_ranking_file = 'HP_M.xlsx'
official_ranking_file_W = 'HP_W.xlsx'

xls_ranking_M = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file)
sheet_names_M = xls_ranking_M.sheet_names

xls_ranking_W = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file_W)
sheet_names_W = xls_ranking_W.sheet_names

len(sheet_names_M)

648

In [ ]:
'''
Search for the ranking data that is the most recent to the given comp_date.
The ranking data comes from the given official athlete ranking list.

Return the ranking data that is closest to the competition date.

This ranking data will be used as benchmark for comparsion to our point system

sheet_names: a list of sheet names represents the ranking data. In order of most recent ranking date, that is most recent to oldest
'''
def search_official_ranking(xls, sheet_names, comp_date):
    for sheet in sheet_names:
        sheet_date = sheet.split('_')[-1]
        # found the ranking data prior to the competition date
        if comp_date > sheet_date:
            return xls.parse(sheet,header = 0)


In [ ]:
'''
Evaluate the system starting from the start_date to validation_end_date.
After evaluate athletes basen on their current competition results and prior Elo scores,
the ranking system update elo scores for all the athletes to fully reflect their true skill based on thier performance.

start_date should be at least > 20131205 because the official published ranking data is only availble after this time

xls_ranking and ranking_sheet_names help the system to find the most recent published offcial ranking
The offcial ranking is used to evaluate the accuracy of the ranking system.

''' 
def evaluate_system(folderName, start_date, validation_end_date, xls_ranking, ranking_sheet_names, show_yearly = False):
    files = os.listdir(folderName)
    # sort files in reverse chronological order 
    files.sort(reverse = False)
    # Go through each competition data
    validation_result = pd.DataFrame(columns=['competition', 'Elo accuracy', 'Official accuracy','Absolute Elo Dev', 'Absolute Official Dev','Pairwise Elo Dev', 'Pairwise Official Dev'])
    testing_result = pd.DataFrame(columns=['competition', 'Elo accuracy', 'Official accuracy','Absolute Elo Dev', 'Absolute Official Dev', 'Pairwise Elo Dev', 'Pairwise Official Dev'])

    start_year = int(str(start_date)[:4])
    # files are in the chronological order from oldest to most recent
    for file in files:
        file_year = int(file.split('_')[-1][:4])
        if file_year < start_year:
            continue                                                                                                                                         
        print('Currently processing file: {}\n'.format(file))
        yearly_result = pd.DataFrame(columns=['competition', 'Elo accuracy', 'Official accuracy','Absolute Elo Dev', 'Absolute Official Dev', 'Pairwise Elo Dev', 'Pairwise Official Dev'])
        if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
            xls = pd.ExcelFile(folderName +'/' + file)
            sheet_names = xls.sheet_names
            #sheet_names.sort(reverse = False)
            
            ## each sheet is a competition in that competition level
            for sheet in sheet_names:
                comp_date = int(sheet.split('_')[-1])
                comp_year = int(sheet.split('_')[-1][0:4])
                # Note: ground truth ranking data starts from 20131205
                # Evaluate the system starting from the start_date
                if comp_date >= start_date:
                  df = xls.parse(sheet,header = 0)
                  # remove the athletes who have not finished the competition
                  df = df[df['Status']== 'QLF']
                  # get the official ranking data
                  official_ranking = search_official_ranking(xls_ranking, ranking_sheet_names, sheet.split('_')[-1])
                  # feed the competition data and official ranking to compute the accuracy
                  elo_accuracy, official_accuracy, elo_deviation, official_deviation, elo_pairwise_deviation, official_pairwise_deviation = evaluation(df, official_ranking)
                  #print('elo: {}, official: {}'.format(elo_accuracy, official_accuracy))
                  yearly_result = yearly_result.append({'competition':sheet,
                                          'Elo accuracy': elo_accuracy,
                                          'Official accuracy': official_accuracy,
                                          'Absolute Elo Dev': elo_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Elo Dev': elo_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True)
                  if comp_date <= validation_end_date:
                      validation_result = validation_result.append({'competition':sheet,
                                          'Elo accuracy': elo_accuracy,
                                          'Official accuracy': official_accuracy,
                                          'Absolute Elo Dev': elo_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Elo Dev': elo_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True)
                  else:
                      testing_result = testing_result.append({'competition':sheet,
                                          'Elo accuracy': elo_accuracy,
                                          'Official accuracy': official_accuracy,
                                          'Absolute Elo Dev': elo_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Elo Dev': elo_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True) 
                  
                  ##### After evaluate the competition, re-calculate participated athletes' Elo
                  # process competition data 
                  competition = competitionFileParser(df, comp_year)
                  # run elo calculation for this competition  
                  calculateElo(competition.competitors, comp_year)
            
            if show_yearly:
                #print(result)
                print(yearly_result.mean())
                print('\n')
    
    print("----------------------------------------------------------------------")
    # here prints out the summarized results for validation and testing periods respectively
    print('Validation results ({} - {})\n'.format(start_date, validation_end_date))
    print(validation_result.mean())
    #print(validation_result)
    print('\n')
    print('Testing results: \n')
    print(testing_result.mean())
    #print(testing_result)
    print('\n')

In [ ]:
def evaluation(df, official_ranking):
    # get all the athletes who participated in the current competition
    participated_athlete = {}
    participated_official_ranking = pd.DataFrame(columns=['fiscode', 'name', 'points'])

    for row in (df.index):
        fis = df.loc[row,'Fis Code']
        # get from Elo system
        existed_athlete = athlete_dict.get(fis)
        # check if this athlete participated in previous competitions
        # Only evaluate on the athleltes who have shown up at least once in the prior competitions
        if existed_athlete:
            participated_athlete[fis] = existed_athlete
        
            # get the athlete data from the offcial ranking data
            official_ranked_athlete = official_ranking[official_ranking['fis code'] == fis]
            # this athlete has not competed in the most recent 52 weeks, which means not on the official ranking. 
            # set Default point to ZERO
            # By default, this athlete loses to all other athletes in the competition
            if official_ranked_athlete.empty:
                participated_official_ranking = participated_official_ranking.append({'fiscode': fis,
                                                                                      'name': existed_athlete.name,
                                                                                      'points':-1}, ignore_index=True)
            else:
                participated_official_ranking = participated_official_ranking.append({'fiscode': fis,
                                                                                      'name': existed_athlete.name,
                                                                                      'points':official_ranked_athlete['points'].values[0]}, ignore_index=True)    
                  
    #sort the official ranking results
    participated_official_ranking = participated_official_ranking.sort_values(by=['points'],ascending=False, ignore_index=True)
    # sort the Elo point system
    participated_elo_ranking = sort_athlete(participated_athlete)
    # print('Official\n')
    # print(participated_official_ranking)
    # print('Elo\n')
    # print(participated_elo_ranking)

    # filter the athletes who have participated in previous competitions
    df_filtered = df[df['Fis Code'].isin(participated_elo_ranking['fiscode'])].loc[:,['Athlete Name', 'Fis Code', 'Rank', 'Race Points']]
    #df_filtered = df.loc[:,['Athlete Name', 'Fis Code', 'Rank', 'Race Points']]

    # pairwise compare every two athletes
    elo_accuracy, official_accuracy = pairwise_accuracy(df_filtered, participated_elo_ranking, participated_official_ranking)
    elo_deviation, official_deviation = deviation_accuracy(df_filtered, participated_elo_ranking, participated_official_ranking)
    elo_pairwise_deviation, official_pairwise_deviation = pairwise_deviation_accuracy(df_filtered, participated_elo_ranking, participated_official_ranking)
    
    return elo_accuracy, official_accuracy, elo_deviation, official_deviation, elo_pairwise_deviation, official_pairwise_deviation


In [ ]:
'''
pairwise compare athletes using Elo ranking and the official ranking with the actual competition ranks
This method evaluates whether one athlete ranks higher than others according to the predicted ranks from Elo and Official data, and compare this prediction to the actual competition result
'''
def pairwise_accuracy(competition_ranking_df, elo_ranking_df, official_ranking_df):    
    count = 0
    elo_correct = 0
    official_correct = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0] - 1):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_elo = elo_ranking_df[elo_ranking_df['fiscode']==athlete1]['elo'].values[0]
        athlete1_official = official_ranking_df[official_ranking_df['fiscode']==athlete1]['points'].values[0]
        for j in range(i+1, competition_ranking_df.shape[0]):
            athlete2 = competition_ranking_df.iloc[j, 1]
            athlete2_elo = elo_ranking_df[elo_ranking_df['fiscode']==athlete2]['elo'].values[0]
            athlete2_official = official_ranking_df[official_ranking_df['fiscode']==athlete2]['points'].values[0]
            if athlete1_elo > athlete2_elo and athlete2_elo != 0:
                elo_correct += 1
            if athlete1_official > athlete2_official and athlete2_official != -1:
                official_correct += 1
            count += 1
    return elo_correct/count, official_correct/count

In [ ]:
'''
Calculate accuracy by comparing the deviation of the competition rank positions from Elo system and Official rankings to the true competition ranks.
This method evaluates one athlete's predicted rank position his/her true ranks based on Elo and Official data

'''
def deviation_accuracy(competition_ranking_df, elo_ranking_df, official_ranking_df):    
    count = 0
    elo_deviation = 0
    official_deviation = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0]):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_elo_rank = elo_ranking_df[elo_ranking_df['fiscode']==athlete1].index[0]
        athlete1_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete1].index[0]
          
        elo_deviation += abs(i - athlete1_elo_rank)
        official_deviation += abs(i - athlete1_official_rank)
        count += 1
    return elo_deviation/count, official_deviation/count

In [ ]:
'''
Calculate accuracy by comparing the pairwise deviation of the rank positions from Elo, Official rankings to the true competition ranks.
This method evaluates the difference between one athlete's predicted rank position to other athletes' and one's true rank to others based on Elo and Official data

'''
def pairwise_deviation_accuracy(competition_ranking_df, elo_ranking_df, official_ranking_df):    
    count = 0
    elo_pairwise_deviation = 0
    official_pairwise_deviation = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0] - 1):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_elo_rank = elo_ranking_df[elo_ranking_df['fiscode']==athlete1].index[0]
        athlete1_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete1].index[0]
        for j in range(i+1, competition_ranking_df.shape[0]):
            athlete2 = competition_ranking_df.iloc[j, 1]
            athlete2_elo_rank = elo_ranking_df[elo_ranking_df['fiscode']==athlete2].index[0]
            athlete2_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete2].index[0]
            
            
            true_diff = i - j
            elo_pairwise_deviation += abs(true_diff - (athlete1_elo_rank - athlete2_elo_rank))
            official_pairwise_deviation += abs(true_diff - (athlete1_official_rank - athlete2_official_rank))
            count += 1
    return elo_pairwise_deviation/count, official_pairwise_deviation/count

In [ ]:
# evaluate the Elo point system up until the end_date
evaluate_system(BASE_DIR+'Female', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_W, ranking_sheet_names= sheet_names_W, show_yearly=True)

Currently processing file: W_2014.xlsx

Elo accuracy             0.775159
Official accuracy        0.557285
Absolute Elo Dev         2.152755
Absolute Official Dev    1.743739
Pairwise Elo Dev         3.282809
Pairwise Official Dev    2.774364
dtype: float64


Currently processing file: W_2015.xlsx

Elo accuracy             0.699622
Official accuracy        0.491235
Absolute Elo Dev         2.301654
Absolute Official Dev    2.229669
Pairwise Elo Dev         3.471859
Pairwise Official Dev    3.468414
dtype: float64


Currently processing file: W_2016.xlsx

Elo accuracy             0.785203
Official accuracy        0.528335
Absolute Elo Dev         2.621372
Absolute Official Dev    2.668550
Pairwise Elo Dev         3.879940
Pairwise Official Dev    4.069453
dtype: float64


Currently processing file: W_2017.xlsx

Elo accuracy             0.748830
Official accuracy        0.742561
Absolute Elo Dev         3.715825
Absolute Official Dev    2.972451
Pairwise Elo Dev         5.610450
Pairwis

In [ ]:
# evaluate the Elo point system up until the end_date
evaluate_system(BASE_DIR+'Male', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_M, ranking_sheet_names= sheet_names_M, show_yearly=True)

Currently processing file: M_2014.xlsx

Elo accuracy             0.705295
Official accuracy        0.372731
Absolute Elo Dev         4.058143
Absolute Official Dev    4.049249
Pairwise Elo Dev         6.052646
Pairwise Official Dev    6.096599
dtype: float64


Currently processing file: M_2015.xlsx

Elo accuracy             0.719219
Official accuracy        0.540202
Absolute Elo Dev         4.388355
Absolute Official Dev    3.926045
Pairwise Elo Dev         6.509467
Pairwise Official Dev    5.947014
dtype: float64


Currently processing file: M_2016.xlsx

Elo accuracy             0.717784
Official accuracy        0.601795
Absolute Elo Dev         4.200753
Absolute Official Dev    3.648601
Pairwise Elo Dev         6.243549
Pairwise Official Dev    5.494012
dtype: float64


Currently processing file: M_2017.xlsx

Elo accuracy             0.690953
Official accuracy        0.645097
Absolute Elo Dev         6.006102
Absolute Official Dev    5.168251
Pairwise Elo Dev         8.790607
Pairwis

## Compute athletes' track of record (Elo scores)




In [ ]:
# organize the athletes' track of records
columns = ['FIS code', 'Name', 'Gender', 'Birthdate', 'First competition year', 'Last competition year' ,'First competition level','Nationality']
columns.extend([i for i in range(0, 30)])

track_records = pd.DataFrame(columns = columns)

for fis in athlete_dict.keys():
    info = athlete_dict[fis].getInfo()[:-1]
    
    # clean the non-participated years 
    records = athlete_dict[fis].records
    start_year = athlete_dict[fis].start_year
    end_year = athlete_dict[fis].end_year
    for i in range (1, end_year-start_year):
        if records[i] == None:
            records[i] = records[i-1] 
    info.extend(records)

    track_records.loc[len(track_records)] = info

In [ ]:
track_records

,FIS code,Name,Gender,Birthdate,First competition year,Last competition year,First competition level,Nationality,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,1021507,Lael GREGORY,M,2/23/1974 12:00:00 AM,1995,1999,WC,USA,2188.477105,2640.36,2722.09,2728.89,2730.37,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1021604,Ivan ZELLER,M,1/1/1970 12:00:00 AM,1995,1996,WC,SUI,2044.937115,2360.31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1018888,Ross POWERS,M,2/10/1979 12:00:00 AM,1995,2006,WC,USA,2140.626177,2691.58,2913.82,3056.17,3227.14,3332.35,3393.88,3395.4,3395.4,3484.17,4073.11,4084.1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,1002204,Mike JACOBY,M,5/20/1969 12:00:00 AM,1995,1996,WC,USA,1860.356385,1967.81,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,1115112,Dustin DEL GUDICE,M,4/23/1974 12:00:00 AM,1995,1997,WC,USA,1931.154058,2307.72,2322.27,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,9531480,Jake LANGSTON,M,7/29/2004 12:00:00 AM,2019,2019,NAC,USA,1376.293711,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4611,9531481,Shayne SANDBLOM,M,4/1/2003 12:00:00 AM,2019,2019,NAC,USA,1340.658950,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4612,9531601,Luccas LEDEZMA,M,9/14/2001 12:00:00 AM,2019,2019,NAC,USA,1340.218233,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4613,9531488,Luke ALLEN,M,9/26/2002 12:00:00 AM,2019,2019,NAC,USA,1338.859707,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
# save the track records to drive
# track_records.to_csv('MIE490: Capstone/Data/records.csv')

## Find podium winner athletes and thier competition records

In [ ]:
'''
Go through all WC or World Champ competition data to record podium winners
Return a DataFrame with information on the competition name/year of the first and last podium as well as all gold medals that athlete won 
'''
def find_champion(folderName, ChampList):
    files = os.listdir(folderName)
    # sort files in chronological order 
    files.sort(reverse = False)
    # Go through each competition data
    for file in files:
        print(file.split('_')[-1])
        if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
            xls = pd.ExcelFile(folderName +'/' + file)
            sheet_names = xls.sheet_names
            sheet_names.sort(reverse = False)
            ## each sheet is a competition in that competition level
            for sheet in sheet_names:
                comp_level = sheet.split('_')[2]
                if comp_level not in ["WC", "WorldChamp"]:
                  continue;
                else:
                  #print("find gold medal from ", sheet)
                  df = xls.parse(sheet,header = 0)
                  df = df[df['Status']!= 'DNS'] 
                  #podium = df["Rank"][df['Rank']<=3]
                  #print(podium)
                  for i in range(1,4):
                    row = df["Rank"][df['Rank']==i].index[0]    
                    fisCode = df.loc[row,'Fis Code']
                    name = df.loc[row,'Athlete Name']
                    country = df.loc[row,'Nation']
                    birthday = df.loc[row,'Birthdate']
                    #print("rank:" , df.loc[row,'Rank'], name," from ", country)
                    
                    if i == 1: #gold
                      if fisCode not in ChampList["fiscode"].values: #first time wining any medal
                        #print("first time winning gold")
                        ChampList = ChampList.append({'fiscode':fisCode, 'Name':name, 'country':country, 'Birthday':birthday, "First Podium":sheet, "Gold Medal #1":sheet}, ignore_index=True)
                      
                      else: # have won medals before
                        #print("athlete has won medals before, got gold again")
                        for c in ChampList.columns[6:]:
                          if pd.isnull(ChampList.loc[ChampList["fiscode"]==fisCode,c].iloc[0]):  #fill the first NaN column
                            ChampList[c][ChampList["fiscode"]==fisCode] = sheet
                            break                            
                          elif c == ChampList.columns[-1] and not pd.isnull(ChampList.loc[ChampList["fiscode"]==fisCode,c].iloc[0]): #create a column
                            #print("create a new column: ", ChampList.columns[-1][:-1] +str(int(ChampList.columns[-1][-1])+1))
                            ChampList[ChampList.columns[-1][:-1] +str(int(ChampList.columns[-1][-1])+1)] = np.nan
                            ChampList[ChampList.columns[-1]][ChampList["fiscode"]==fisCode] = sheet
                      ChampList["Last Gold"][ChampList["fiscode"]==fisCode] = sheet
                    else: #silver or broze medal
                      if fisCode not in ChampList["fiscode"].values:
                        ChampList = ChampList.append({'fiscode':fisCode, 'Name':name, 'country':country, 'Birthday':birthday, "First Podium":sheet, "Gold Medal #1":np.nan}, ignore_index=True)
                    ChampList["Last Podium"][ChampList["fiscode"]==fisCode] = sheet  
            #print(ChampList)
    return ChampList
  
#train_system(BASE_DIR+'Male', end_date = 20191231)
columns=['fiscode', 'Name', 'country', 'Birthday',"First Podium", "Last Podium","Last Gold","Gold Medal #1"]
ChampList = pd.DataFrame(columns=columns)
ChampList = find_champion(BASE_DIR+'Male', ChampList)
ChampList.to_excel('MIE490: Capstone/Data/M_ChampionList.xlsx')

ChampList_W = pd.DataFrame(columns=columns)
ChampList_W = find_champion(BASE_DIR+'Female', ChampList_W)
ChampList_W.to_excel('MIE490: Capstone/Data/W_ChampionList.xlsx')

## Hyperparameter tuning

In [ ]:
_MAXSIZE = 80

_MINBUFFEREDELO = 1000
_MAXBUFFEREDELO = 3000
_KMAX_list = [45]
_BUFFEREDELOSLOPE = -.28 / 1300
_EXPODENTIALRATE = 800
_WIN = 1.0
_LOSS = 0.0
_DEFAULT_K_list = [40]
_MAXDENOM = 1/((_KMAX - _DEFAULT_K)/(3*_MAXSIZE))


for a, b in zip(_KMAX_list, _DEFAULT_K_list):
    _KMAX = a
    _DEFAULT_K = b
    # _EXPODENTIALRATE = rate
    # print("EXPODENTIALRATE is {}\n".format(rate))
    print('KMAX is {}, Default K is {}'.format(_KMAX, _DEFAULT_K))
    
    ### Reset GLOBAL VARIABLE
    # initialize the dictionary to store all athletes data
    athlete_dict = {}

    # train the Elo point system up until the end_date
    train_system(BASE_DIR+'Female', end_date = 20131231 )

    # train the Elo point system up until the end_date
    evaluate_system(BASE_DIR+'Female', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_W, ranking_sheet_names= sheet_names_W, show_yearly=True)
    print('\n')

KMAX is 45, Default K is 40
Currently processing file: W_2014.xlsx

Elo accuracy             0.744840
Official accuracy        0.557285
Absolute Elo Dev         2.200866
Absolute Official Dev    1.743739
Pairwise Elo Dev         3.352010
Pairwise Official Dev    2.774364
dtype: float64


Currently processing file: W_2015.xlsx

Elo accuracy             0.687772
Official accuracy        0.491235
Absolute Elo Dev         2.280729
Absolute Official Dev    2.229669
Pairwise Elo Dev         3.465286
Pairwise Official Dev    3.468414
dtype: float64


Currently processing file: W_2016.xlsx

Elo accuracy             0.810166
Official accuracy        0.526717
Absolute Elo Dev         2.337139
Absolute Official Dev    2.657700
Pairwise Elo Dev         3.444379
Pairwise Official Dev    4.070335
dtype: float64


Currently processing file: W_2017.xlsx

Elo accuracy             0.757961
Official accuracy        0.743287
Absolute Elo Dev         3.580935
Absolute Official Dev    2.985367
Pairwise Elo 

In [ ]:
_MAXSIZE = 80

_MINBUFFEREDELO_list = [800, 1000]
_MAXBUFFEREDELO_list = [2500, 3000, 3500, 4000, 4500]
_KMAX_list = [ 30, 35]
_BUFFEREDELOSLOPE = -.28 / 1300
_EXPODENTIALRATE = 800
_WIN = 1.0
_LOSS = 0.0
_DEFAULT_K_list = [25, 28]
_MAXDENOM = 1/((_KMAX - _DEFAULT_K)/(3*_MAXSIZE))


for a, b in zip(_KMAX_list, _DEFAULT_K_list):
    _KMAX = a
    _DEFAULT_K = b
    # _EXPODENTIALRATE = rate
    # print("EXPODENTIALRATE is {}\n".format(rate))
    print('KMAX is {}, Default K is {}'.format(_KMAX, _DEFAULT_K))


    for min_buf in _MINBUFFEREDELO_list:
        for max_buf in _MAXBUFFEREDELO_list:
            _MINBUFFEREDELO = min_buf
            _MAXBUFFEREDELO = max_buf
            print("min buf: {}, max buf: {}".format(_MINBUFFEREDELO, _MAXBUFFEREDELO))
    
            
            ### Reset GLOBAL VARIABLE
            # initialize the dictionary to store all athletes data
            athlete_dict = {}

            # train the Elo point system up until the end_date
            train_system(BASE_DIR+'Male', end_date = 20131231 )

            # train the Elo point system up until the end_date
            evaluate_system(BASE_DIR+'Male', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_M, ranking_sheet_names= sheet_names_M, show_yearly=False)
            print('\n')

KMAX is 30, Default K is 25
min buf: 800, max buf: 2500
Currently processing file: M_2014.xlsx

Currently processing file: M_2015.xlsx

Currently processing file: M_2016.xlsx

Currently processing file: M_2017.xlsx

Currently processing file: M_2018.xlsx

Currently processing file: M_2019.xlsx

----------------------------------------------------------------------
Validation results (20140101 - 20171231)

Elo accuracy             0.707205
Official accuracy        0.552475
Absolute Elo Dev         4.669246
Absolute Official Dev    4.296808
Pairwise Elo Dev         6.910402
Pairwise Official Dev    6.488594
dtype: float64


Testing results: 

Elo accuracy             0.730727
Official accuracy        0.450429
Absolute Elo Dev         4.147592
Absolute Official Dev    3.821824
Pairwise Elo Dev         6.165191
Pairwise Official Dev    5.780237
dtype: float64




min buf: 800, max buf: 3000
Currently processing file: M_2014.xlsx

Currently processing file: M_2015.xlsx

Currently processing

In [ ]:
_MAXSIZE = 80

_MINBUFFEREDELO = 1000
_MAXBUFFEREDELO = 3000
_KMAX_list = [35]
_BUFFEREDELOSLOPE = -.28 / 1300
_EXPODENTIALRATE_list = [800, 1000, 1200, 1500]
_WIN = 1.0
_LOSS = 0.0
_DEFAULT_K_list = [28]
_MAXDENOM = 1/((_KMAX - _DEFAULT_K)/(3*_MAXSIZE))


for a, b in zip(_KMAX_list, _DEFAULT_K_list):
    _KMAX = a
    _DEFAULT_K = b
    print('KMAX is {}, Default K is {}'.format(_KMAX, _DEFAULT_K))
    for rate in _EXPODENTIALRATE_list:
        _EXPODENTIALRATE = rate
        print('exp rate: {}'.format(rate))
        # _EXPODENTIALRATE = rate
        # print("EXPODENTIALRATE is {}\n".format(rate))
        
        
        ### Reset GLOBAL VARIABLE
        # initialize the dictionary to store all athletes data
        athlete_dict = {}

        # train the Elo point system up until the end_date
        train_system(BASE_DIR+'Male', end_date = 20131231 )

        # train the Elo point system up until the end_date
        evaluate_system(BASE_DIR+'Male', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_M, ranking_sheet_names= sheet_names_M, show_yearly=False)
        print('\n')      

KMAX is 35, Default K is 28
exp rate: 800
Currently processing file: M_2014.xlsx

Currently processing file: M_2015.xlsx

Currently processing file: M_2016.xlsx

Currently processing file: M_2017.xlsx

Currently processing file: M_2018.xlsx

Currently processing file: M_2019.xlsx

----------------------------------------------------------------------
Validation results (20140101 - 20171231)

Elo accuracy             0.710721
Official accuracy        0.552475
Absolute Elo Dev         4.598605
Absolute Official Dev    4.296808
Pairwise Elo Dev         6.853672
Pairwise Official Dev    6.488594
dtype: float64


Testing results: 

Elo accuracy             0.733792
Official accuracy        0.450429
Absolute Elo Dev         4.124475
Absolute Official Dev    3.821824
Pairwise Elo Dev         6.188384
Pairwise Official Dev    5.780237
dtype: float64




exp rate: 1000
Currently processing file: M_2014.xlsx

Currently processing file: M_2015.xlsx

Currently processing file: M_2016.xlsx

Current